## Spark Notebook Setup

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}

import $ivy.$
import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs

Getting spark JARs

Creating SparkSession

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@39ff85d5

## Preprocessing Data

In [4]:
val moviePath = "notebooks/data/movies.csv"
val stopWordsPath = "notebooks/data/stopwords.txt"
val lemmatizationPath = "notebooks/data/lemmatization.txt"

moviePath: String = "notebooks/data/movies.csv"
stopWordsPath: String = "notebooks/data/stopwords.txt"
lemmatizationPath: String = "notebooks/data/lemmatization.txt"

In [5]:
val moviesRaw = spark.read
    .option("header", "true")
    .option("multiline", "true")
    .option("escape", "\"")
    .csv(moviePath)
    .rdd
    .zipWithIndex
    .map(x => (x._2, x._1.getString(0), x._1.getString(1), x._1.getString(7))) // (id, title, genres)

moviesRaw.take(5).foreach(println)

csv at cmd5.sc:5

take at cmd5.sc:10

(0,1901,Kansas Saloon Smashers,A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1])
(1,1901,Love by the Light of the Moon,The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything

moviesRaw: org.apache.spark.rdd.RDD[(Long, String, String, String)] = MapPartitionsRDD[9] at map at cmd5.sc:8

In [6]:
val stopWords = spark.read
    .textFile(stopWordsPath)
    .rdd
    .map(_.trim.toLowerCase.replaceAll("[^a-z]", ""))
    .collect()
    .toSet

collect at cmd6.sc:4

stopWords: Set[String] = Set(
  "quotient",
  "snow",
  "liquid",
  "eye",
  "down",
  "wash",
  "rub",
  "side",
  "measure",
  "please",
  "trouble",
  "finger",
  "read",
  "number",
  "mass",
  "able",
  "behind",
  "mother",
  "chief",
  "hurry",
  "for",
  "drive",
  "magnet",
  "tire",
  "find",
  "neighbor",
  "wild",
  "support",
  "produce",
  "school",
  "rock",
  "soldier",
  "weight",
  "question",
  "art",
  "parent",
  "shop",
  "method",
...

In [7]:
val lemmatization = spark.read
    .textFile(lemmatizationPath)
    .rdd
    .map(_.split("\\s+"))
    .map(x => (x(1), x(0)))
    .collect()
    .toMap

lemmatization.take(5).foreach(println)

collect at cmd7.sc:5

(professed,profess)
(pathogens,pathogen)
(purifies,purify)
(phosphates,phosphate)
(buns,bun)


lemmatization: Map[String, String] = Map(
  "professed" -> "profess",
  "pathogens" -> "pathogen",
  "purifies" -> "purify",
  "phosphates" -> "phosphate",
  "buns" -> "bun",
  "fathering" -> "father",
  "soapiest" -> "soapy",
  "basils" -> "basil",
  "showdowns" -> "showdown",
  "redcurrants" -> "redcurrant",
  "night-watchmen" -> "night-watchman",
  "regularizing" -> "regularize",
  "boutiques" -> "boutique",
  "satsumas" -> "satsuma",
  "healings" -> "healing",
  "breaks" -> "break",
  "cut-backs" -> "cut-back",
  "sneezed" -> "sneeze",
  "forgotten" -> "forget",
  "insinuates" -> "insinuate",
  "stocktakings" -> "stocktaking",
  "cartwheeled" -> "cartwheel",
  "boxers" -> "boxer",
  "dynasties" -> "dynasty",
  "hydrologists" -> "hydrologist",
  "sectors" -> "sector",
  "radiopharmaceuticals" -> "radiopharmaceutical",
  "embedded" -> "embed",
  "rationalising" -> "rationalise",
  "respecting" -> "respect",
  "exorcists" -> "exorcist",
  "allotments" -> "allotment",
  "styrenes" -> "

In [8]:
val movies = moviesRaw.map { case (id, year, title, plot) =>
    val processedPlot = plot.toLowerCase
        .replaceAll("[^a-z ]", " ")
        .split(" ")
        .filterNot(stopWords.contains)
        .filterNot(_.isEmpty)
        .map(token => lemmatization.getOrElse(token, token))

    (id, year, title, processedPlot)
}

movies.take(5).foreach(println)

take at cmd8.sc:12

(0,1901,Kansas Saloon Smashers,[Ljava.lang.String;@4d663cac)
(1,1901,Love by the Light of the Moon,[Ljava.lang.String;@6bd453e3)
(2,1901,The Martyred Presidents,[Ljava.lang.String;@2529deb1)
(3,1901,Terrible Teddy, the Grizzly King,[Ljava.lang.String;@36e79761)
(4,1902,Jack and the Beanstalk,[Ljava.lang.String;@498e9b6a)


movies: org.apache.spark.rdd.RDD[(Long, String, String, Array[String])] = MapPartitionsRDD[21] at map at cmd8.sc:1

In [9]:
val vocabulary = movies 
    .flatMap { case (_, _, _, cleanedPlot) => cleanedPlot }
    .distinct

vocabulary.take(10).foreach(println)

distinct at cmd9.sc:2

take at cmd9.sc:5

mistretta
shh
sammee
bone
gerven
nothin
rostom
mislabel
fred
bresac


vocabulary: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[25] at distinct at cmd9.sc:2

In [10]:
val vocabularyIndexed = vocabulary
    .zipWithIndex
    .map(_.swap)

vocabularyIndexed.take(10).foreach(println)

take at cmd10.sc:5

(0,mistretta)
(1,shh)
(2,sammee)
(3,bone)
(4,gerven)
(5,nothin)
(6,rostom)
(7,mislabel)
(8,fred)
(9,bresac)


vocabularyIndexed: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[27] at map at cmd10.sc:3

In [11]:
def dfToIdf(df: Int, totalDocs: Long): Double = {
    math.log(totalDocs.toDouble / df)
}

defined function dfToIdf

In [12]:
val numDocs = movies.count
val vocabularySize = vocabularyIndexed.count.toInt

count at cmd12.sc:1

count at cmd12.sc:2

numDocs: Long = 34886L
vocabularySize: Int = 112805

In [13]:
val vocabIdfs = movies
    .flatMap { case (_, _, _, cleanedPlot) => cleanedPlot }
    .map { word => (word, 1) }
    .reduceByKey(_ + _)
    .zipWithIndex
    .map { case ((word, count), id) => (word, id, count) }
    .map { case (word, id, count) => (word, id, dfToIdf(count, numDocs)) }

vocabIdfs.take(10).foreach(println)

map at cmd13.sc:3

take at cmd13.sc:9

(mistretta,0,8.513930732522766)
(shh,1,10.45984088157808)
(sammee,2,10.45984088157808)
(bone,3,5.250354728736659)
(gerven,4,10.45984088157808)
(nothin,5,9.36122859290997)
(rostom,6,8.061945608779709)
(mislabel,7,9.36122859290997)
(fred,8,3.370597726550566)
(bresac,9,10.45984088157808)


vocabIdfs: org.apache.spark.rdd.RDD[(String, Long, Double)] = MapPartitionsRDD[33] at map at cmd13.sc:7

In [14]:
import org.apache.spark.rdd.RDD
import $ivy.`org.apache.spark::spark-mllib:2.4.0`
import org.apache.spark.mllib.linalg.{Vector, Vectors}

import org.apache.spark.rdd.RDD
import $ivy.$
import org.apache.spark.mllib.linalg.{Vector, Vectors}

In [15]:
def tokenListToTfIdf(tokenList: List[String], vocabulary: RDD[(String, Long, Double)]): List[Double] = {
    val tfs = spark.sparkContext.parallelize(tokenList)
        .map((_, 1))
        .reduceByKey(_ + _)

    val idfs = vocabulary 
        .map { case (word, id, idf) => (word, (id, idf)) }

    val tfidfs = tfs.rightOuterJoin(idfs)
        .map { case (word, (tfOpt, (id, idf))) => (word, tfOpt.getOrElse(0), idf, id) }
        .map { case (word, tf, idf, id) => (id, tf * idf) }
        .sortBy(_._1)
        .collect
        .map(_._2)
        .toList

    tfidfs
}

defined function tokenListToTfIdf

In [15]:
// val tokenList = List("apple", "banana", "orange", "apple")
// val vocabulary = spark.sparkContext.parallelize(Seq(("apple", 0, 2), ("banana", 1, 2), ("orange", 2, 1)))

In [15]:
// val vocabIdfs = vocabulary
//     .map { case (word, id, df) => (word, id, dfToIdf(df, 3)) }
// 
// val tf = tokenListToTfIdf(tokenList, vocabIdfs)
// println("test", tf)

In [16]:
val firstEntry = movies.first()

first at cmd16.sc:1

firstEntry: (Long, String, String, Array[String]) = (
  0L,
  "1901",
  "Kansas Saloon Smashers",
  Array(
    "bartender",
    "work",
    "saloon",
    "serve",
    "drink",
    "customer",
    "fill",
    "stereotypically",
    "irish",
    "s",
    "bucket",
    "beer",
    "carrie",
    "follower",
    "burst",
    "inside",
    "assault",
    "irish",
    "pull",
    "eye",
    "dump",
    "beer",
    "wreck",
    "smash",
    "fixture",
    "mirror",
    "break",
    "cash",
    "register",
    "bartender",
    "spray",
    "seltzer",
    "s",
    "policeman",
...

In [17]:
vocabIdfs.take(10).foreach(println)

take at cmd17.sc:1

(mistretta,0,8.513930732522766)
(shh,1,10.45984088157808)
(sammee,2,10.45984088157808)
(bone,3,5.250354728736659)
(gerven,4,10.45984088157808)
(nothin,5,9.36122859290997)
(rostom,6,8.061945608779709)
(mislabel,7,9.36122859290997)
(fred,8,3.370597726550566)
(bresac,9,10.45984088157808)


In [18]:
val firstEntryTfIdf = tokenListToTfIdf(firstEntry._4.toList, vocabIdfs)
println(firstEntryTfIdf)

map at cmd15.sc:3

map at cmd15.sc:7

sortBy at cmd15.sc:12

sortBy at cmd15.sc:12

collect at cmd15.sc:12

List(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

firstEntryTfIdf: List[Double] = List(
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
...

In [19]:
println(firstEntryTfIdf.sum)

136.8393980256314


In [ ]:
def countTokens(tokens: List[String]): Map[String, Int] = {
    tokens.groupBy(identity).mapValues(_.size)
}

defined function countTokens

In [31]:
val tokenCounts = movies
    .flatMap { case (id, title, genres, plotTokens) => {
        val tfs = plotTokens
            .groupBy(identity)
            .mapValues(_.size)
            .map { case (token, count) => (token, id, count) }
            .toList
        tfs
    }}

tokenCounts.take(10).foreach(println)

take at cmd31.sc:11

(eye,0,1)
(register,0,1)
(s,0,2)
(wreck,0,1)
(break,0,1)
(serve,0,1)
(burst,0,1)
(everybody,0,1)
(carrie,0,1)
(dump,0,1)


tokenCounts: RDD[(String, Long, Int)] = MapPartitionsRDD[95] at flatMap at cmd31.sc:2

In [32]:
vocabIdfs.take(10).foreach(println)

take at cmd32.sc:1

(mistretta,0,8.513930732522766)
(shh,1,10.45984088157808)
(sammee,2,10.45984088157808)
(bone,3,5.250354728736659)
(gerven,4,10.45984088157808)
(nothin,5,9.36122859290997)
(rostom,6,8.061945608779709)
(mislabel,7,9.36122859290997)
(fred,8,3.370597726550566)
(bresac,9,10.45984088157808)
